# An Improved Model For Predicting Property Tax Assessed Values

---

## Project Goals

Identify attributes that can be used to predict property tax assessed values, build a prediction model that is an improvement upon the existing model, and offer recommendations on what works or doesn't work.

---

## Project Description

The Zillow data science team would like to predict property tax assessed values for single family properties that had a transaction in 2017. A model already exists, but the Zillow data science team is hoping an outside perspective can lead to an improved model. We will analyze the data available and produce an improved prediction model. We will compare the new model to the existing one and offer recommendations on what works or doesn't work for acquiring the most accurate predictions.

---

## Initial Questions

Before beginning a few initial questions were formulated from an initial high level overview of the data available. Here are the questions that this project aimed to answer:

- What influence do bedroom count, bathroom count, and square footage have on property value?
- Is there a relationship between the age of a property and its value?
- Is there a relationship between the number of stories a property has and its value?
- Is there a relationship between the construction type of a property and its value?
- Does a basement increase a property's value?
- Is there a relationship between the a building's class and its value?
- Is there a relationship between heating or system type of a property and its value?
- Is there a relationship between the age of a property and its value?
- Does location relate to a property's value?
- Is there a relationship between the architectural style of a property and its value?

---

## Importing the Required Modules

Everything we need to run the code blocks in this report are located in the util folder.
To run the code blocks in this report you will need numpy, pandas, matplotlib, seaborn and sklearn installed on your computer.

In [ ]:
# Here we are importing everything needed to execute the code blocks in this notebook.
from util.acquire import wrangle_zillow

---

## Data Acquisition and Preparation

We acquire the data from data.codeup.com using a SQL query. Acquiring the data takes some time so it is cached in a local "telco.csv" file for quicker access in the future. The "telco.csv" file is not stored in the Github repository.



In [ ]:
# Here we will acquire, prepare, and split the zillow property data and store the results in our train, validate, 
# and test dataframes.
train, validate, test = wrangle_zillow()

---

## Exploratory Analysis

---

## Modeling

---

## Key Takeaways and Recommendations